In [1]:
import numpy as np
import pandas as pd

Попробуем с помощью утилиты `wget` получить данные по ссылке на датасет на Гитхабе. Поскольку Колабовские машины работают на Linux, это должно сработать.